# PART I: CREATE THE REQUIRED DATA FRAME

### READ THE WIKI PAGE WITH BEAUTIFULSOUP

In [1]:
# import the library we use to open URLs
import urllib.request

In [2]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [4]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [5]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
# print(soup.prettify())

In [6]:
right_table=soup.find('table', class_='wikitable sortable')

In [7]:
### CREATE LIST AND APPEND ROWS INTO THE DATA FRAME

In [8]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append((cells[0].find(text=True)).strip())
        B.append((cells[1].find(text=True)).strip())
        C.append((cells[2].find(text=True)).strip())

In [9]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postalcode'])
df['Borough']=B
df['Neighbourhood']=C

### FORMAT THE DATA FRAME 

In [10]:
# Remove any rows with not assigned Borough 
df = df[df['Borough'] != 'Not assigned']

# Join the Neighbourhood for all rows with same postal code then reset index
df = df.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Update the neighbourhood value with the corresponding neighbourhood name if it is not initially assigned
df.loc[df['Neighbourhood']=='Not assigned','Neighbourhood']=df['Borough']

# PART II: ADD THE LATITUDE AND LONGITUDE ATTRIBUTE 

### Update the coordinates using Geocoder package (not with provided csv file)

In [11]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
lat_coords = []
long_coords = []

# loop until you get the coordinates
for index, row in df.iterrows():
    while(lat_lng_coords is None):
        g = geocoder.arcgis(('{}, ' + row['Borough'] + ', CA').format(row['Postalcode']))
        lat_lng_coords = g.latlng
        lat_coords.append(lat_lng_coords[0])
        long_coords.append(lat_lng_coords[1])
    lat_lng_coords = None

In [12]:
df['Latitude'] = lat_coords
df['Longitude'] = long_coords

pd.set_option('display.max_rows', None)
df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967


# PART III: CLUSTER NEIGHBOURHOOD BASED ON DATA READ FROM FOURSQUARE

### Filter data with only Borough in Toronto area

In [13]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [14]:
import folium # map rendering library

# create map of Toronto using latitude 43.6 and longitude -79.4 value
latitude = 43.65
longitude = -79.4

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = 'IILM0AQZ5XZZXZ1B2PUPCAXUY5FEF5MJHFUMMSIS1SYEPLWY' # your Foursquare ID
CLIENT_SECRET = 'XYO2SAZ5XQ5DSJGH5LLOG4E1AWQSHKGDY2U0J3C0TDZWDXWW' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IILM0AQZ5XZZXZ1B2PUPCAXUY5FEF5MJHFUMMSIS1SYEPLWY
CLIENT_SECRET:XYO2SAZ5XQ5DSJGH5LLOG4E1AWQSHKGDY2U0J3C0TDZWDXWW


In [16]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67653121600006, -79.29542499999997.


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=IILM0AQZ5XZZXZ1B2PUPCAXUY5FEF5MJHFUMMSIS1SYEPLWY&client_secret=XYO2SAZ5XQ5DSJGH5LLOG4E1AWQSHKGDY2U0J3C0TDZWDXWW&v=20180604&ll=43.67653121600006,-79.29542499999997&radius=500&limit=100'

In [18]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",68,68,68,68,68,68
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 142 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,...,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
1,Berczy Park,0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.00,0.029412,0.014706,0.00000,0.000000,0.000000,0.000000,...,0.014706,0.000000,0.000000,0.014706,0.000000,0.00,0.029412,0.014706,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.030000,0.00,0.010000,0.010000,0.02000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.030000,0.020000,0.000000,0.00,0.010000,0.000000,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
5,"Cabbagetown, St. James Town",0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
6,Central Bay Street,0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
8,Christie,0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01
9,Church and Wellesley,0.000000,0.030000,0.00,0.010000,0.000000,0.03000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.010000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.01


In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Breakfast Spot,Burger Joint,Gastropub,Bar,Bakery,Asian Restaurant
1,Berczy Park,Coffee Shop,Café,Hotel,Steakhouse,Breakfast Spot,Burger Joint,Gastropub,Bar,Bakery,Asian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Bakery,Sandwich Place,Hotel,Beer Bar,Supermarket,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Hotel,Steakhouse,Sushi Restaurant,American Restaurant,Pizza Place,Pub,Thai Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Hotel,Steakhouse,Breakfast Spot,Burger Joint,Gastropub,Bar,Bakery,Asian Restaurant


In [27]:
### APPLY THE K-MEANS CLUSTERING ALGORITHM

In [28]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,3,Trail,Health Food Store,Pub,Women's Store,Diner,Ethiopian Restaurant,Elementary School,Electronics Store,Eastern European Restaurant,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683178,-79.355105,2,Grocery Store,Park,Bus Line,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Cupcake Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314667,0,Sandwich Place,Gym,Ice Cream Shop,Pub,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse,Liquor Store,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0,Diner,Italian Restaurant,Café,Pizza Place,Bar,Sushi Restaurant,Coffee Shop,Sandwich Place,Brewery,Arts & Crafts Store
4,M4N,Central Toronto,Lawrence Park,43.609727,-79.492844,1,Park,Convenience Store,Grocery Store,Tennis Court,Bank,Breakfast Spot,Gay Bar,Gastropub,Gourmet Shop,Gluten-free Restaurant


In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters